In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "san_francisco" dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_trips" table
table_ref = dataset_ref.table("bikeshare_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


In [4]:
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                      ORDER BY trip_date
                  """

# Run the query, and return a pandas DataFrame
num_trips_result = client.query(num_trips_query).result().to_dataframe()
print(num_trips_result)

      trip_date  num_trips  cumulative_trips
0    2015-01-01        181               181
1    2015-01-02        428               609
2    2015-01-03        283               892
3    2015-01-04        206              1098
4    2015-01-05       1186              2284
..          ...        ...               ...
360  2015-12-27        146            344692
361  2015-12-28        475            345167
362  2015-12-29        502            345669
363  2015-12-30        319            345988
364  2015-12-31        264            346252

[365 rows x 3 columns]
